# Préparation du jeu de données PubChem - Ajout du bruit++ (Tesla)

À ce stade, les réseaux que l'on entraîne sont capables de prédire le bruit ajouté de manière relativement efficace. En effet, lorsqu'on ajoute un bruit de RMSE 28, les réseaux actuels sont capables de le prédire avec un RMSE de 18. Cela revient à se tromper en moyenne à 18 mÅ soit 0.018 Å sur les distances de chaque atome aux atomes fictifs placés dans le repère. Il s'agit donc d'une prédicition qui n'est pas idéale, mais qui semble tout de même raisonnable pour obtenir une molécule "plus optimisée".

Pour mieux évaluer les modèles que l'on produit dans des cas plus extrêmes, nous allons introduire un bruit plus élevé dans les coordonnées. L'idée est que si le modèle produit de bonnes prédictions avec plus de bruit, il sera également efficace sur des données possédant moins de bruit. Le bruit que nous allons introduire va déplacer les atomes de l'ordre de 0.3 Å.

Nous allons de plus calculer de façon automatique le RMSE du bruit introduit lors de la génération des données bruitées, afin d'avoir une mesure comparative pour évaluer la performance des modèles.

Enfin, nous allons procéder à une augmentation des données d'entraînement, en passant trois fois sur les molécules du jeu de données original et en ajoutant un bruit différent à chaque fois. Nous allons donc passer de 3,7 millions de molécules à 11,1 millions. Cela permettra de limiter le nombre d'époques d'entraînement (apprentissage sur toutes les données) nécessaires et donc de limiter le risque de sur-apprentissage.

#### Définition des chemins de fichiers

In [1]:
test_set_location = "../data/test_set_riken_v2.h5"
test_set_prepared_input_location = "../data/test_set_riken_v2_prepared_input_bruit+.h2"
test_set_labels_location = "../data/test_set_riken_v2_labels_bruit+.h2"

train_set_location = "../data/train_set_riken_v2.h5"
train_set_prepared_input_location = "../data/train_set_riken_v2_prepared_input_bruit+.h5"
train_set_labels_location = "../data/train_set_riken_v2_labels_bruit+.h5"

minimal_set_riken_location = "../data/minimal_set_riken_v2.h5"
minimal_set_prepared_input_location = "../data/minimal_set_riken_v2_prepared_input_bruit+.h5"
minimal_set_labels_location = "../data/minimal_set_riken_v2_labels_bruit+.h5"

mini_set_riken_location = "../data/mini_set.h5"
mini_set_prepared_input_location = "../data/mini_set_prepared_input_bruit+.h5"
mini_set_labels_location = "../data/mini_set_labels_bruit+.h5"



#### Définition de la fonction d'ajout de bruit

In [2]:
import numpy as np

def positions_bruitees(positions):    
    bruit = np.random.normal(loc=0.0, scale=0.1732, size=positions.shape)
    return ((positions + bruit), bruit)



#### Définition de la fonction de calcul de la matrice de distances compressée à partir de la matrice des coordonnées des atomes

In [3]:
def matrice_distances_compr(positions):
    """ Renvoie la matrice de distances compressée des positions des atomes passées en paramètres
    La matrice de distances compressée est définie de la façon suivante : pour chaque atome, on calcule
    la distance avec chaque point du repère. Une ligne i de la matrice (n,4) correspond aux distances
    de l'atome i avec chacun des quatre points du repère"""
    
    nb_at = len(positions)
    
    # On renvoie un tableau vide si la molécule est vide
    if nb_at == 0:
        return []
    
    repere = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
    repere = np.vstack([repere]*nb_at)

    positions = np.tile(positions, 4).reshape(4*nb_at, 3)
    
    return np.sqrt(np.sum(np.power(positions-repere, 2), 1)).reshape(nb_at, 4)


#### Fonction de création des entrées et des labels des RN

In [4]:
import h5py
import time
import numpy as np

def creation_input_RN(set_location, input_rn_location, labels_location, epochs):
    
    start_time = time.time()    
    mol_vides = 0
    
    rmse_epochs = []
    
    print("Creating input and label sets for "+set_location+" : ")

    # On charge le jeu de données original (en lecture seule)
    original_dataset_h5 = h5py.File(set_location, 'r')
    
    # On enregistre la taille du jeu de données
    taille = len(original_dataset_h5["anums"])
    
    # On créé les jeux de données d'entrée du RN et de labels
    input_rn_dataset_h5 = h5py.File(input_rn_location, 'w')
    labels_dataset_h5 = h5py.File(labels_location, 'w')
    
    
    # On créé les datasets inputs et targets 
    input_dataset = input_rn_dataset_h5.create_dataset("inputs", shape=(epochs*taille, 1000),
                                       dtype=np.float32, compression="gzip", 
                                       chunks=True)

    targets_dataset = labels_dataset_h5.create_dataset("targets", shape=(epochs*taille, 800),
                                       dtype=np.float32, compression="gzip", 
                                       chunks=True)

    
    try:
        
        for k in range(epochs):

            # Contient les rmse de tous les bruits ajoutés à chaque molécule durant toute l'époque de génération
            # de l'entrée du RN. À la fin de l'époque, on en fait la moyenne et on l'ajoute à rmse_epochs
            bruits_epoch_rmse = np.empty(shape=(taille,), dtype=np.float32)
            
            np_input_dataset = np.empty(shape=(taille, 1000))
            np_targets_dataset = np.empty(shape=(taille, 800))
            
            input_coords = np.array(original_dataset_h5["riken_coords"])
            input_masses = np.array(original_dataset_h5["amasses"])
            print("loaded input")

            # On parcourt toutes les molécules de l'exemple
            for i in range(taille):

                if i%100 == 0:
                    print("Computing input and label sets for epoch "+str(k)+" and molecule "+str(i)+" ("+str(i/taille*100)+"%)")

                
                # On récupère les coordonnées de la molécule courante et on y ajoute du bruit
                coords = np.array(input_coords[i]).reshape(-1,3)
                dist_init = matrice_distances_compr(coords)
                coords_bruit, bruit = positions_bruitees(coords)
                coords_bruit = coords_bruit.reshape(-1, 3)


                bruit = bruit.reshape(-1, 3)

                if len(coords) == 0:
                    mol_vides += 1
                else:
                    # On calcule les différence de distances cibles (en mÅ) et les distances bruitées (en Å)
                    dist_bruit = matrice_distances_compr(coords_bruit)
                    delta_dist_targets = (dist_init - dist_bruit)*1000

                # On récupère les masses atomiques de la molécule courante
                masses = input_masses[i]

                # On initialise l'entrée du RN et le vecteur cible pour la molécule courante
                entree_courante = np.zeros(shape=(1000, 1))
                cible_courante = np.zeros(shape=(200, 4))

                # On ajoute les coordonnées bruitées et les masses à l'entrée avec padding, et les coordonnées
                # cibles au dataset targets
                j=0
                for masse in masses:

                    # Ajout des données au vecteur entrée
                    index_input_courant = j*5
                    entree_courante[index_input_courant] = dist_bruit[j][0]
                    entree_courante[index_input_courant+1] = dist_bruit[j][1]
                    entree_courante[index_input_courant+2] = dist_bruit[j][2]
                    entree_courante[index_input_courant+3] = dist_bruit[j][3]
                    entree_courante[index_input_courant+4] = masse

                    # Ajout des données à la matrice cibles
                    cible_courante[j] = delta_dist_targets[j]

                    j+=1

                # On aplatit le vecteur cibles
                cible_courante = cible_courante.reshape(1, 800)

                # On insère les données dans le tableau np en mémoire
                np_input_dataset[i] = entree_courante.reshape(-1, 1000)
                np_targets_dataset[i] = cible_courante

                # On ajoute le rmse ajouté à la molécule au tableau des rmse de l'époque
                np_delta_dist_targets = np.array(delta_dist_targets)
                bruits_epoch_rmse[i] = np.sqrt(np.mean(np.square(np_delta_dist_targets)))
                
            # On ajoute le rmse moyen de l'époque au tableau des rmse de toutes les époques
            rmse_epochs.append(np.mean(bruits_epoch_rmse))
            
            # On écrit toutes les données de l'époque dans le fichier h5 en mémoire
            print("Writing data to h5 for epoch "+str(k))
            input_dataset[k*taille: k*taille+taille] = np_input_dataset
            targets_dataset[k*taille: k*taille+taille ] = np_targets_dataset
          

        print("Writing datasets to disk")
        input_rn_dataset_h5.flush()
        labels_dataset_h5.flush()
        
        print("--- %s seconds ---" % (time.time() - start_time))
        
        # On calcule le rmse moyen de toutes les époques
        np_rmse_epochs = np.array(rmse_epochs)
        print("RMSE bruit moyen : "+str(np.mean(np_rmse_epochs)))
        
    finally:
        original_dataset_h5.close()
        input_rn_dataset_h5.close()
        labels_dataset_h5.close()
    



/home/etudiant/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
creation_input_RN(mini_set_riken_location, mini_set_prepared_input_location,
                  mini_set_labels_location, 3)

Creating input and label sets for ../data/mini_set.h5 : 
loaded input
Computing input and label sets for epoch 0 and molecule 0 (0.0%)
Writing data to h5 for epoch 0
loaded input
Computing input and label sets for epoch 1 and molecule 0 (0.0%)
Writing data to h5 for epoch 1
loaded input
Computing input and label sets for epoch 2 and molecule 0 (0.0%)
Writing data to h5 for epoch 2
Writing datasets to disk
--- 0.035329341888427734 seconds ---
RMSE bruit moyen : 182.75244


In [75]:
total_start_time = time.time()

#### Préparation des données d'entrée du RN et des labels pour le jeu minimal

In [76]:
print()
print("Préparation données jeu minimal :")
creation_input_RN(minimal_set_riken_location, minimal_set_prepared_input_location,
                  minimal_set_labels_location, 3)


Préparation données jeu minimal :
Creating input and label sets for ../data/minimal_set_riken_v2.h5 : 


MemoryError: 

##### Sortie : 
```
RMSE bruit moyen : 171.73126
```

#### Préparation des données d'entrée du RN et des labels pour le jeu d'entraînement

In [ ]:
print()
print("Préparation données jeu d'entraînement :")
creation_input_RN(train_set_location, train_set_prepared_input_location, train_set_labels_location, 3)

##### Sortie :
```
RMSE bruit moyen : 171.7593
```

#### Préparation des données d'entrée du RN et des labels pour le jeu de validation

In [ ]:
print()
print("Préparation données jeu de validation :")
creation_input_RN(test_set_location, test_set_prepared_input_location, test_set_labels_location, 3)

In [13]:
print("Total time : ")
print("--- %s seconds ---" % (time.time() - total_start_time))

Total time : 
--- 138.90174651145935 seconds ---
